In [38]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [39]:
df = pd.read_csv('IphoneData.csv')
newdf = df.drop(columns=['NAME', 'CO2E'])
features = newdf.to_numpy()
emissions = df['CO2E'].to_numpy()

In [104]:
def modelScore(X, y, model):
    avgR2 = avgSSE = avgMAPE = 0
    #cross validation using K folds
    kFolds = 5
    for i, (trainIndex, testIndex) in enumerate(KFold(n_splits=kFolds, shuffle=True, random_state=42).split(X)):
        X_train = X[trainIndex]
        X_test = X[testIndex]
        y_train = y[trainIndex]
        y_test = y[testIndex]
    

        if model == 'linear regression':
            fittedModel = LinearRegression().fit(X_train, y_train)
                
        elif model == 'decision tree':
            fittedModel = tree.DecisionTreeRegressor().fit(X_train, y_train)
                
        elif model == 'random forest':
            y_train = y_train.flatten()
            RF = RandomForestRegressor(max_features='log2', n_estimators=200, max_depth=20)
            fittedModel = RF.fit(X_train, y_train)
            
        elif model == 'support vector regression':
            y_train = y_train.flatten()
            fittedModel = SVR(kernel = 'linear').fit(X_train, y_train)
    
        elif model == 'polynomial regression':
            polyX_train = PolynomialFeatures(degree=2).fit_transform(X_train)
            polyX_test = PolynomialFeatures(degree=2).fit_transform(X_test)
            fittedModel = LinearRegression().fit(polyX_train, y_train)
            X_test = polyX_test

        elif model == 'xgboost':
            xgbModel = xgb.XGBRegressor(gamma=0, learning_rate=0.2, max_depth=7, min_child_weight=1,
                                        n_estimators=200)
            fittedModel = xgbModel.fit(X_train, y_train)
        
        elif model =='lasso':
            fittedModel = Lasso(alpha=0.1, selection='random', tol=0.019, max_iter=2000).fit(X_train, y_train)
            

        y_pred = fittedModel.predict(X_test)

        #scores
        avgR2 += r2_score(y_test, y_pred)    
        avgSSE += np.sum((y_test - y_pred) ** 2)
        avgMAPE += mean_absolute_percentage_error(y_test, y_pred)

    avgR2, avgSSE, avgMAPE = avgR2 / kFolds, avgSSE / kFolds, avgMAPE / kFolds
   #print results
    print(model)
    print("R2 = " + str(avgR2))
    print("SSE = " + str(avgSSE))
    print("MAPE = " + str(avgMAPE))




In [41]:
modelScore(features, emissions, 'linear regression')

linear regression
R2 = 0.9132785915590607
SSE = 172.6195696345183
MAPE = 0.04614202622279183


In [42]:
modelScore(features, emissions, 'decision tree')

decision tree
R2 = 0.6971522333469551
SSE = 630.2
MAPE = 0.08856735190391377


In [96]:
modelScore(features, emissions, 'random forest')

random forest
R2 = 0.7346807975454197
SSE = 627.696604431
MAPE = 0.08351883748287627


In [44]:
modelScore(features, emissions, 'support vector regression')

support vector regression
R2 = 0.8783182219644387
SSE = 231.72982845335702
MAPE = 0.0474385532925264


In [45]:
modelScore(features, emissions, 'polynomial regression')

polynomial regression
R2 = 0.5193065980378624
SSE = 1344.874736766754
MAPE = 0.07132996748624869


In [98]:
modelScore(features, emissions, 'xgboost')

xgboost
R2 = 0.8169820514834409
SSE = 413.21905292950396
MAPE = 0.06661729974262168


In [106]:
modelScore(features, emissions, 'lasso')

lasso
R2 = 0.9016827063666252
SSE = 201.59638137772362
MAPE = 0.049656795514902395


In [107]:
modelsList = ['linear regression', 'decision tree', 'random forest', 'support vector regression', 'polynomial regression', 'xgboost', 'lasso']
multipleFeatures = df[['DISPLAY (inch)', 'STORAGE (gb)', 'MEMORY (gb)', 'CPU (cores)', 'GPU (cores)', 'NEURAL ENGINE (cores)', 'WEIGHT (gm)', 'FRONT CAMERA (mp)', ]].to_numpy()
featuresList = ['DISPLAY (inch)', 'STORAGE (gb)', 'MEMORY (gb)', 'CPU (cores)', 'GPU (cores)', 'NEURAL ENGINE (cores)', 'WEIGHT (gm)', 'FRONT CAMERA (mp)', 'BATTERY (mAh)', 'number of camera (back)', 'Sum of megapixels', 'MAIN CAMERA (mp)']

In [108]:
for model in modelsList:
    modelScore(multipleFeatures, emissions, model)

linear regression
R2 = 0.8652429792464236
SSE = 269.9701058383988
MAPE = 0.0590561116474827
decision tree
R2 = 0.7368416344419012
SSE = 522.6
MAPE = 0.07675398092049404
random forest
R2 = 0.8022469594512792
SSE = 475.0003264888889
MAPE = 0.07086934128844433
support vector regression
R2 = 0.845590618898018
SSE = 323.64650039570176
MAPE = 0.05645566907054338
polynomial regression
R2 = -3.267953668841885
SSE = 12402.994123520346
MAPE = 0.14423965319490623
xgboost
R2 = 0.7790930758465069
SSE = 461.0258527153317
MAPE = 0.07193249772831659
lasso
R2 = 0.8706637314066581
SSE = 278.7100612813084
MAPE = 0.058110862003714445


In [50]:
"""for x in featuresList:
    print(x)
    for model in modelsList:
        print(modelScore(df[x].to_numpy().reshape(-1, 1), emissions, model))
    print("")  """

'for x in featuresList:\n    print(x)\n    for model in modelsList:\n        print(modelScore(df[x].to_numpy().reshape(-1, 1), emissions, model))\n    print("")  '